## Feature Analysis for machine learning

In [ ]:
# Loading necessary modlules
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import seaborn as sns
from pprint import pprint
pl.style.use('seaborn-ticks')
import warnings
from sklearn.utils import resample
from sources.permutation_importance import* 

# Machine learning models
from sklearn.ensemble import RandomForestClassifier
# Machine Learning models
rf_model = RandomForestClassifier()

In [ ]:
# loading the data from the machine

# mightee_data = pd.read_csv('old-test-train/raw_data_original.csv')
mightee_data = pd.read_csv('normalised/scaled_raw_zs.csv')
# roc_data = pd.read_csv('normalised/roc_data.csv')

X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')


X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [ ]:
# X_train

In [ ]:
print('The number of All', len(y_train))

print('The number of True/AGN', sum(np.array(y_train['labels'] == 1)))

In [ ]:
mightee_data.describe()
# X_train.describe()

## CORRELATION MATRIX

In [ ]:
#Using Pearson Correlation, ### You dont have to seperate the sources in a correlation matrix its called data leakage
pl.figure(figsize=(12,10))
cor = mightee_data.corr()
sns.heatmap(cor, annot=True, cmap=pl.cm.Reds)
pl.show()

In [ ]:
## Making a pair plot using seaborn
import seaborn as sns
sns.set(font_scale=2)
sns.set_style('ticks')
sns.pairplot(mightee_data, hue="class_labels", 
             plot_kws = { 'edgecolor': 'k', 'alpha':0.4, 'lw':0.5, 's':20 }, size = 4)


## Sequential Feature importance

In [ ]:
y_tr = y_train['labels']
y_t = y_test['labels']

# Permutation Feature Importance

In [ ]:
mean_importances, std_importances = perm_impotance(X_train, y_tr, rf_model)
# mean_importances, std_importances = perm_impotance(X_balanced, y_balanced, rf_model)



importance1 = manual_mean_importances(mean_importances)
std1 = manual_mean_importances(std_importances)
column_names = np.array(X_train.columns)

sorted_idx1 = importance1.argsort()
sorted_idx1 = sorted_idx1[::-1]

x_pos = np.arange(len(column_names)) #- 1)  # -1 for indexs
x_class = column_names[sorted_idx1]
print("column names: ", x_pos)

In [ ]:
X_train.columns

In [ ]:
print("column names: ", x_class)

In [ ]:
importance1[sorted_idx1]

In [ ]:
std1[sorted_idx1]

In [ ]:
x_pos

In [ ]:
# Build the plot
fig, ax = pl.subplots(figsize = (12, 9))
ax.bar(x_pos, importance1[sorted_idx1], yerr=std1[sorted_idx1], align='center', alpha=0.5, color = 'green', ecolor='black', capsize=10)
ax.set_ylabel('Importance', fontsize=25)
ax.set_xticks(x_pos)
# ax.set_xticklabels(x_class, ha='right', rotation = 45, fontsize = 16, fontweight ='bold')
# ax.set_xticklabels(x_class, fontsize = 25, fontweight ='bold')
#First confirm the order of features from x_class
new_colnames = [r'$q_\mathrm{IR}$', 
                'class_star',
                r'$log (S_{8.0}/S_{4.5})$',
                r'$log (S_{5.8}/S_{3.6})$',
                r'$log (M_{\rm star})$',
                r'$log (S_{4.5}/S_{3.6})$'
               ]
# ax.set_xticklabels(x_class, fontsize = 25)
ax.set_xticklabels(new_colnames, fontsize = 25)


# Rotate the tick labels and set their alignment.
pl.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Save the figure and show
pl.tight_layout()
pl.savefig('perm_importance.pdf')
pl.show()

## ROC Feature Analysis

In [ ]:
from sklearn.metrics import *
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
y = y_train['labels']
y_t = y_test['labels']

In [ ]:
# do not touch
def threshold (X, Y, direction):
    acc = []
    tp = []     
    fp = []
    
    th = np.array(X).flatten()
    
    inTrain, outTrain = (list(t) for t in zip(*sorted(zip(th, np.array(y).flatten()))))
    
    thresholds = np.arange(inTrain[0], inTrain[-1]+1 , .0001)
    print(inTrain[0])
    print(inTrain[-1]+1)
    
    for i in thresholds:
        pred = []

        for xTr in inTrain:
            if direction:
                if i > xTr:
                    pred.append(1)
                else:
                    pred.append(0)
            else:
                if i < xTr:
                    pred.append(1)
                else:
                    pred.append(0)

#         print(pred)        
        acc.append(accuracy_score(outTrain, pred))
        CM = confusion_matrix(outTrain, pred)

        TN = CM[0][0]
        FN = CM[1][0]
        TP = CM[1][1]
        FP = CM[0][1]
        
        TPR = TP/(TP+FN)
        TNR = TN/(TN+FP) 
        tp.append(TPR)
        fp.append(1-TNR)
        
#         acc.append(accuracy_score(outTest, predTest))        
#         thre.append(i)
        
        
    return acc, fp, tp

# colours = [red, blue, green, dark yellow, grey, ]
colors = [ "blue", 'red', 'green', 'grey', 'orange', 'purple'] 
def draw_ROC (di, features, labels, X, y, colors):
    lw = 2
    pl.figure(figsize=(10 , 8))
    for f, label, d, color in zip(features, labels, di, colors):
        x = np.array(X[[f]]).flatten()
        # x = np.array(X_balanced[[f]]).flatten() #new test for balanced data
        y = np.array(y).flatten()
        acc, tp, fp = threshold (x, y, d)
        print()
        pl.plot(tp, fp, lw=2, color=color, label = label+" AUC: "+str(round(metrics.auc(tp, fp),2)) )
        
    pl.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    pl.xlim([0.0, 1.0])
    pl.ylim([0.0, 1.05])
    # plt.title('$MeOH$', fontweight ='bold', fontsize =12)
    pl.xlabel('False Positive Rate', fontsize =20)
    pl.ylabel('True Positive Rate', fontsize =20)
    # pl.title('ROC curves based on features', fontweight ='bold', fontsize =18)
    pl.legend(loc="lower right", prop={"size":16})
    pl.savefig('normalised/roc_importance.pdf')
    pl.show() 

In [ ]:
# column names
# feat = X_train.columns 
feat = ['qir', 'Mstar', 'class_star', 'log(S8/S45)', 'log(S45/S36)'] #, 'log(S58/S36)' ]

# These are column used only during the plotting
# label = [r'$q_\mathrm{IR}$',
#          r'$log (M_{\rm star})$', 
#          'class_star',
#          r'$log (S_{8.0}/S_{4.5})$',
#          r'$log (S_{5.8}/S_{3.6})$',
#          r'$log (S_{4.5}/S_{3.6})$'
#                ]
label = [r'$q_\mathrm{IR}$',
         r'$log (M_{\rm star})$', 
         'class_star',
         r'$log (S_{8.0}/S_{4.5})$',
         # r'$log (S_{4.5}/S_{3.6})$',
         r'$log (S_{5.8}/S_{3.6})$'
               ]
# the direction of the feature
di_feat = [False, True, True, False, True, True]

# call ROC fuction
draw_ROC (di_feat, feat, label, X_train, y, colors=colors) #Original
# draw_ROC (di_feat, feat, label, y=y_balanced, colors=colors)
# draw_ROC (di_feat, feat, label, X_roc, y, colors=colors)


## Random Forest Importances

In [ ]:
feature_names = X_train.columns

rf_model.fit(X_train, y_tr)


import time

start_time = time.time()
rf_importances = rf_model.feature_importances_
rf_std = np.std([tree.feature_importances_ for tree in rf_model.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")
sorted_idxrf = rf_importances.argsort()
# sorted_idxrf = rf_importances.sort(reverse=True)
sorted_idxrf = sorted_idxrf[::-1]
x_pos_rf = np.arange(len(feature_names))
x_class_rf = feature_names[sorted_idxrf]

In [ ]:
print("Correct cols order: ", x_class_rf) 

In [ ]:
# Build the plot
fig, ax = pl.subplots(figsize = (12, 9))
ax.bar(x_pos_rf, rf_importances[sorted_idxrf], yerr= rf_std[sorted_idxrf], align='center', alpha=0.5, color = 'green', ecolor='black', capsize=10)
ax.set_ylabel('Importance', fontsize=25)
ax.set_xticks(x_pos_rf)
# ax.set_xticklabels(x_class_rf, fontsize = 25,fontweight ='bold')
new_colnames_rf = [r'$q_\mathrm{IR}$', 
                r'$log (S_{8.0}/S_{4.5})$',
                'class_star',
                r'$log (S_{5.8}/S_{3.6})$',
                r'$log (M_{\rm star})$',
                r'$log (S_{4.5}/S_{3.6})$'
               ]
ax.set_xticklabels(new_colnames_rf, fontsize = 25)
# Rotate the tick labels and set their alignment.
pl.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
# Save the figure and show
pl.tight_layout()
pl.savefig('rf_importance.pdf')
pl.show()

In [ ]:
feature_names

In [ ]:
print

In [ ]:
print